In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [17]:
mainPath1 = './templates/contents/contents_1/귀어귀촌개념.html'
mainPath2 = './templates/contents/contents_1/귀어귀촌준비절차.html'
mainPath3 = './templates/contents/contents_1/어업면허허가.html'
mainPath4 = './templates/contents/contents_1/어업정의.html'
mainPath5 = './templates/contents/contents_2/귀어정책지원교육.html'
mainPath6 = './templates/contents/contents_2/양식장및어선임대사업.html'
mainPath7 = './templates/contents/contents_2/어촌이사혜택.html'
mainPath8 = './templates/contents/contents_2/청년어촌정착지원금.html'

file_path = [
            # mainPath1
            # ,mainPath2
            # ,mainPath3
            # ,mainPath4
            #, mainPath5
            #, mainPath6
            #, mainPath7
            mainPath8
             ]
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "\n", ".", " ", ""],  # 끊는 우선순위
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(strip=True)

                content = contents[k].get_text(strip=True)
                

                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(strip=True)
            content = contents[0].get_text(strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print("제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df



,title,content,folder,page
0,청년어촌정착지원금,청년 어촌 정착 지원사업은 창업 초기 청년 수산업 경영인의 안정적인 어촌 정착을 돕...,contents_2,청년어촌정착지원금


In [45]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("faiss_store/contents")

C:\Users\human\AppData\Local\Temp\ipykernel_5956\4096798401.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
